In [1]:
using Pkg
Pkg.activate("climate"; shared=true)

"/Users/imolk/Library/Julia/environments/climate/Project.toml"

In [2]:
ICAO = "KWRB"
USAF, WBAN = 722175, 13860
k_nearest = 5

5

In [3]:
using Revise
import TempModel

┌ Info: Recompiling stale cache file /Users/imolk/Library/Julia/compiled/v0.7/TempModel/5V5SX.ji for TempModel [429a4ede-8e99-57b3-891e-c3971593d88c]
└ @ Base loading.jl:1185


In [4]:
data_dir = "../data"
epsg = 3857 # Web Mercator (m)
isdList = TempModel.read_isdList(; data_dir=data_dir, epsg=epsg)
isd_wData = TempModel.stations_with_data(isdList; data_dir=data_dir)
isd_nearest_and_test = TempModel.find_nearest(isd_wData, USAF, WBAN, k_nearest)
isd_nearest = isd_nearest_and_test[2:end,:]

hourly_data = TempModel.read_Stations(isd_nearest; data_dir=data_dir)
;

In [5]:
k_spatiotemporal, logNoise = k_spatiotemporal = TempModel.fitted_sptemp_matern()
;

In [6]:
@time opt_out = TempModel.optim_kernel(k_spatiotemporal, logNoise, isd_nearest, hourly_data, :Optim);

20880.

  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41
  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41


152039 seconds (28.81 M allocations: 17.438 GiB, 0.08% gc time)


In [8]:
opt_out

Dict{Symbol,Any} with 4 entries:
  :mll      => -72132.8
  :hyp      => [-1.55879, -0.13913, 1.29157, 13.5471, -1.52905, -0.767335, -0.3…
  :logNoise => -1.55879
  :opt_out  => Results of Optimization Algorithm…

In [9]:
import JSON
save_dir = "."
GPmodel = "matern"
output_dictionary = Dict{String,Any}(
    "mll" => opt_out[:mll],
    "hyp" => opt_out[:hyp],
    "logNoise" => opt_out[:logNoise],
    "test_ICAO" => ICAO,
    "test_USAF" => USAF,
    "test_WBAN" => WBAN,
    "nearby_ICAO" => isd_nearest[:ICAO],
    "nearby_USAF" => isd_nearest[:USAF],
    "nearby_WBAN" => isd_nearest[:WBAN],
    "GPmodel" => GPmodel
)

savemodel_dir = joinpath(save_dir, "fitted_kernel", GPmodel)
if !isdir(savemodel_dir)
    mkdir(savemodel_dir)
end
fname = @sprintf("hyperparams_%s_%s.json", GPmodel, ICAO) 
filepath = joinpath(savemodel_dir, fname)
open(filepath, "w") do io
    indent = 4
    JSON.print(io, output_dictionary, indent)
end

Add `using Printf` to your imports.
  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41


In [28]:
k1.kernel

Type: GaussianProcesses.Periodic, Params: [-0.13913, 1.29157, 3.17805]

In [29]:
ksum, k_st = k_spatiotemporal.kleft, k_spatiotemporal.kright
ksum, k_st = ksum.kleft, ksum.kright
k4, ksp4 = k_st.kleft, k_st.kright
k4 = k4.kernel
ksp4 = ksp4.kernel.kernel

ksum, k_st = ksum.kleft, ksum.kright
k3, ksp3 = k_st.kleft, k_st.kright
k3 = k3.kernel
ksp3 = ksp3.kernel.kernel

ksum, k_st = ksum.kleft, ksum.kright
k2, ksp2 = k_st.kleft, k_st.kright
k2 = k2.kernel
ksp2 = ksp2.kernel.kernel

k_st = ksum
k1, ksp1 = k_st.kleft, k_st.kright
k1 = k1.kernel.kernel
ksp1 = ksp1.kernel.kernel

print("k₁: Periodic \n=================\n")
@printf("σ: %5.3f\n", √k1.σ2)
@printf("l: %5.3f\n", √k1.ℓ2)
@printf("p: %5.0f hours\n", k1.p)
print("> spatial decay:\n")
@printf("l: %5.3f km\n", ksp1.ℓ / 1000)
print("\nk₂: RQIso \n=================\n")
@printf("σ: %5.3f\n", √k2.σ2)
@printf("l: %5.3f hours\n", √ k2.ℓ2)
@printf("α: %5.3f\n", k2.α)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp2.σ2)
@printf("l: %5.3f km\n", ksp2.ℓ / 1000)
print("\nk₃: SEIso \n=================\n")
@printf("σ: %5.3f\n", √k3.σ2)
@printf("l: %5.3f hours\n", √k3.ℓ2)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp3.σ2)
@printf("l: %5.3f km\n", ksp3.ℓ / 1000)
print("\nk₄: RQIso \n=================\n")
@printf("σ: %5.3f\n", √k4.σ2)
@printf("l: %5.3f days\n", √k4.ℓ2 / 24)
@printf("α: %5.3f\n",  k4.α)
print("> spatial decay:\n")
# @printf("σ: %5.3f\n", √ksp4.σ2)
@printf("l: %5.3f km\n", ksp4.ℓ / 1000)
print("\n=================\n")
@printf("σy: %5.3f\n", exp(opt_out[:hyp][1]))

k₁: Periodic 
σ: 3.638
l: 0.870


Add `using Printf` to your imports.
  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41
Add `using Printf` to your imports.
  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41


p:    24 hours
> spatial decay:
l: 764.606 km

k₂: RQIso 
σ: 0.464
l: 0.217 hours
α: 0.712
> spatial decay:
l: 14.107 km

k₃: SEIso 
σ: 1.010
l: 1.742 hours
> spatial decay:
l: 36.377 km

k₄: RQIso 
σ: 3.984
l: 0.302 days
α: 0.272
> spatial decay:
l: 593.225 km

σy: 0.210


Add `using Printf` to your imports.
  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41
Add `using Printf` to your imports.
  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41
Add `using Printf` to your imports.
  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41
Add `using Printf` to your imports.
  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41
Add `using Printf` to your imports.
  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41
Add `using Printf` to your imports.
  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41
Add `using Printf` to your imports.
  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41
Add `using Printf` to your imports.
  likely near /Users/imolk/Library/Julia/packages/IJulia/8Y05k/src/kernel.jl:41
Add `using Printf` to your imports.
  likely near /Users/imolk/Library/J